In [39]:
import pandas as pd
import numpy as np
import seaborn as sbs
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import requests
import json

In [43]:
apikey = json.load(open('alphavantage_key.json'))["key"]

In [2]:
df_tickers = pd.read_excel('tickers/tickers_only.xlsx')

In [6]:
apple = yf.Ticker(df_tickers['Ticker'].values[1])

In [7]:
apple_history = apple.history(start='2000-01-01', end='2024-01-26')

In [26]:
apple_history['Date'] = apple_history.index

In [100]:
apple_history.to_csv('apple_history.csv', index=False)

In [96]:
def extract(ticker:str, start_date:str):
    current_date = datetime.now().strftime('%Y-%m-%d')
    yf_ticker = yf.Ticker(ticker)
    history = yf_ticker.history(start = start_date, end = current_date)
    history['Date']=history.index
    return history

In [97]:
def load(filename: str, ticker: str):
    saved_df = pd.read_csv(filename)
    last_date = datetime.strptime(saved_df['Date'].values[-1][:10], "%Y-%m-%d")
    needed_date = (last_date + timedelta(days=1)).strftime('%Y-%m-%d')
    update_df = extract(ticker, needed_date)
    updated_df = pd.concat([saved_df, update_df], axis=0)
    updated_df = updated_df.reset_index()
    updated_df = updated_df.drop('index', axis=1)
    updated_df.to_csv(filename, index=False)
    return updated_df

In [94]:
load('apple_history.csv', df_tickers['Ticker'].values[1])

2024-01-25 00:00:00-05:00


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
0,0.793754,0.851464,0.769629,0.847207,535796800,0.0,0.0,2000-01-03 00:00:00-05:00
1,0.819298,0.837273,0.765845,0.775779,512377600,0.0,0.0,2000-01-04 00:00:00-05:00
2,0.785239,0.836800,0.779563,0.787131,778321600,0.0,0.0,2000-01-05 00:00:00-05:00
3,0.803215,0.809837,0.719014,0.719014,767972800,0.0,0.0,2000-01-06 00:00:00-05:00
4,0.730367,0.764426,0.722799,0.753073,460734400,0.0,0.0,2000-01-07 00:00:00-05:00
...,...,...,...,...,...,...,...,...
6050,192.300003,195.330002,192.259995,193.889999,60133900,0.0,0.0,2024-01-22 00:00:00-05:00
6051,195.020004,195.750000,193.830002,195.179993,42355600,0.0,0.0,2024-01-23 00:00:00-05:00
6052,195.419998,196.380005,194.339996,194.500000,53631300,0.0,0.0,2024-01-24 00:00:00-05:00
6053,195.220001,196.270004,193.110001,194.169998,54822100,0.0,0.0,2024-01-25 00:00:00-05:00


In [1]:
import tensorflow as tf

In [2]:
print('Tensorflow Version: ' + tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Tensorflow Version: 2.10.0
Num GPUs: 2


In [ ]:
def build_model():
    model = tf.keras.model.Sequential()
    return model

In [16]:
import psycopg2
from psycopg2 import sql

In [28]:
conn = psycopg2.connect(
    host='localhost',
    database='stocks',
    port=5432,
    user='job_bot',
    password='very_secret_password'
)

In [24]:
df1 = pd.read_csv('tickers/tickers_for_db.csv')

In [25]:
df1['ticker_id']=df1.index

In [64]:
def insert_tickers(conn, df1):
    with conn.cursor() as cursor:
        conn.autocommit = True
        tuples = [tuple(x) for x in df1.to_numpy()]
        cols = ','.join(df1.columns)
        query  = "INSERT INTO %s(%s) VALUES(%%s,%%s)" % ('public."Tickers"', cols)
        cursor.executemany(query, tuples)

def insert_companies(conn, apikey):
    with conn.cursor() as cursor:
        cursor.execute('SELECT * FROM public."Tickers" ORDER BY ticker_id ASC')
        for row in cursor:
            idx = row[0]
            ticker = row[1]
            url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol='+ticker+'&apikey='+apikey
            r = requests.get(url)
            data = r.json()
            cols1 = ['AssetType', 'Name', 'Description', 'Exchange', 'Currency', 'Country', 'Sector', 'Industry']
            cols1_db = ['asset_type', 'name', 'description', 'exchange', 'currency', 'country', 'sector', 'industry', 'company_id', 'symbol_id', ]
            cols2_db = ['company_num_id', 'company_id', 'div_per_share', 'div_Yield', '52_week_high', '52_week_low', '50_day_mov_avg', '200_day_mov_avg', 'div_date', 'ex_div_date']
            print(data)
            tuple1 = tuple([data[x] for x in cols1].append[idx, idx])
            #tuple2 = tuple(idx, idx, data[cols2[2:]])


In [11]:
conn.close()

In [22]:
tuples = [tuple(x) for x in df1.to_numpy()]

In [30]:
tuples

[('APPL', 0),
 ('AMZN', 1),
 ('MSFT', 2),
 ('NVDA', 3),
 ('GOOGL', 4),
 ('NFLX', 5),
 ('AMD', 6),
 ('META', 7),
 ('TSLA', 8),
 ('MCD', 9),
 ('VOD', 10),
 ('INTC', 11),
 ('CSCO', 12),
 ('IBM', 13),
 ('QCOM', 14),
 ('TSM', 15),
 ('MU', 16),
 ('SMCI', 17),
 ('NKE', 18),
 ('SBUX', 19),
 ('V', 20),
 ('DIS', 21),
 ('WMT', 22),
 ('COST', 23),
 ('CROX', 24),
 ('SKX', 25),
 ('ADDYY', 26),
 ('GPS', 27),
 ('GES', 28),
 ('MUFG', 29),
 ('SZKMY', 30),
 ('RNLSY', 31),
 ('ISUZY', 32),
 ('MZDAF', 33),
 ('NSANF', 34)]

In [65]:
insert_companies(conn, apikey)

{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


KeyError: 'AssetType'